In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import plot_model

In [3]:
dataset = pd.read_csv("obesity.csv")
print("Classes in the dataset:", dataset['NObeyesdad'].unique())

Classes in the dataset: ['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']


In [4]:
print(dataset.columns)

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')


In [7]:
categories = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'CAEC', 'CALC', 'MTRANS', 'NObeyesdad']
for c in categories:
  dummies = pd.get_dummies(dataset[c], prefix=(str(c)+"_"))
  dataset = pd.concat([dataset, dummies], axis=1)
dataset = dataset.drop(columns=categories)

In [8]:
d = ['NObeyesdad__Insufficient_Weight',
       'NObeyesdad__Normal_Weight', 'NObeyesdad__Obesity_Type_I',
       'NObeyesdad__Obesity_Type_II', 'NObeyesdad__Obesity_Type_III',
       'NObeyesdad__Overweight_Level_I', 'NObeyesdad__Overweight_Level_II']
X = dataset.drop(columns=d)
y = dataset[d]

In [9]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = X.columns)

In [10]:
data_train, data_test, class_train, class_test = train_test_split(X, y, test_size=0.2)
print(data_train.shape)
print(class_train.shape)

(1688, 31)
(1688, 7)


In [11]:
#Neural Network
def createModel():
  model = Sequential()
  model.add(Dense(128, activation='relu', input_dim = 31))
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(7, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
model = createModel()
model.fit(data_train, class_train, epochs=300, batch_size=50)

c:\Users\aadhi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1653 - loss: 2.0003
Epoch 2/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3171 - loss: 1.7779
Epoch 3/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4131 - loss: 1.5755
Epoch 4/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4488 - loss: 1.4116
Epoch 5/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4901 - loss: 1.3247
Epoch 6/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5003 - loss: 1.2578
Epoch 7/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5583 - loss: 1.1554
Epoch 8/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5761 - loss: 1.0895
Epoch 9/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5772 - loss: 1.0824
Epoch 10/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6157 - loss: 0.9862
Epoch 11/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6290 - loss: 0.9475
Epoch 12/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy:

In [12]:
# Accuracy and cost after updating
test_loss, test_acc = model.evaluate(data_test, class_test, verbose=0)
print('Loss:', test_loss)
print('Accuracy:', test_acc)

Loss: 0.2609870135784149
Accuracy: 0.9432623982429504


In [19]:
dataset_copy = pd.read_csv("obesity.csv")

In [20]:
#Cross Validation for DNN - https://stackoverflow.com/questions/48085182/cross-validation-in-keras
from sklearn.model_selection import StratifiedKFold
def train_evaluate(model, xtrain, ytrain, xtest, ytest):
  model.fit(data_train, class_train, epochs=300, batch_size=50)
  test_loss, test_acc = model.evaluate(data_test, class_test, verbose=0)
  return test_loss, test_acc
kFold = StratifiedKFold(n_splits = 10)
i=0
sum=0
temp = dataset_copy["NObeyesdad"]
for train, test in kFold.split(X,temp):
  print()
  model = None
  model = createModel()
  loss, acc = train_evaluate(model,X.iloc[[train[0]]],y.iloc[[train[0]]],X.iloc[[test[0]]],y.iloc[[test[0]]])
  i+=1
  sum += acc

print("Mean accuracy:", (sum/i))
#Mean accuracy: 0.9489361643791199


Epoch 1/300


c:\Users\aadhi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1608 - loss: 1.9701
Epoch 2/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3223 - loss: 1.7764
Epoch 3/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3885 - loss: 1.5882
Epoch 4/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4532 - loss: 1.4215
Epoch 5/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4892 - loss: 1.2997
Epoch 6/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5301 - loss: 1.2260
Epoch 7/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5574 - loss: 1.1632
Epoch 8/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5505 - loss: 1.1533
Epoch 9/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6144 - loss: 1.0648
Epoch 10/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5967 - loss: 1.0362
Epoch 11/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6226 - loss: 0.9868
Epoch 12/300
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6403 - lo